# Pipeline
This notebook takes LCO data straight from the [Archive](https://archive.lco.global), placed into the `data` directory, and performs photometry on the supernova within the images. Optionally, this can perform difference imaging to remove the background galaxy.

To run the notebook step-by step, click on the code cell below, and press `Shift-Enter` on your keyboard to run the cells in sequence. Output files will arrive in the `output` directory after you've finished - you can download the PDF files to see how the photometry went.

In [ ]:
import glob
import logging
import os
import shutil

logging.getLogger().setLevel("INFO")


def cleanup():
    psf_outputs = ["*_seq.txt", "template_*.fits"]
    if os.path.exists("PSF_output/"):
        shutil.rmtree("PSF_output/")

    for output in psf_outputs:
        if outlist := glob.glob(output):
            for item in outlist:
                os.remove(item)

    if fitslist := glob.glob("*[.fits,.fz]"):
        for file in fitslist:
            os.remove(file)


# Clean up any old runs
cleanup()

# Copy frames from data to here - temporarily.
alldata = glob.glob("../data/*[.fits,.fz]")

local_paths = []

for file in alldata:
    potential_path = os.path.join(os.getcwd(), os.path.basename(file))
    local_paths.append(potential_path)

    if not os.path.exists(potential_path):
        shutil.copy(file, os.getcwd())

# Unpack the FITS files - these are compressed by default to save bandwidth while transmitting.
if any(os.path.splitext(f)[-1] == ".fz" for f in alldata):
    logging.info("Unpacking FITS files")
    !funpack -D *.fz

### Configuring the pipeline
We need to set a few things before running the pipeline:
- `TARGET_RA`, `TARGET_DEC`: the right ascension and declination of the supernova, so we know where it should be in the image. If you're running this in Colab, you can use the boxes that 

In [ ]:
# Here we need to set some parameters, so the code knows where to look
TARGET_RA = 155.457239894  # @param {type:"number"}
TARGET_DEC = 56.9279057494  # @param {type:"number"}

# To enable image subtraction, add `--sub` to the list of options below
# To enable cosmic ray cleaning, add `--clean` to the list of options below

!python \
    ../photometry-sans-frustration/psf.py \
    -c $TARGET_RA $TARGET_DEC \
    --clean  \
    --quiet \
    --savefigs;

In [ ]:
logging.info("Cleaning up and placing things in correct place")

for path in local_paths:
    os.remove(path)

# move the PSF output to the output folder
for file in glob.glob("PSF_output/*"):
    shutil.move(file, "../outputs/")

cleanup()